# Baseline Performance

How good are predictions based on climatology, persistence or COSMO-7 alone?

In [ ]:
import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from plots import retrieval_template, statistical_eval
from db_tools import read_csv_profiles, read_csv_mean

%matplotlib inline

plt.rcParams["font.family"] = "DejaVu Sans"

# Data Preparation

- Radiosonde climatology
- COSMO-7 forecasts with 2 different lead times

In [ ]:
T_raso = read_csv_profiles("../data/unified/test/T_rasoclim.csv")

T_0006 = read_csv_profiles("../data/unified/priors/T_cosmo7+00+06_mean.csv")
T_2430 = read_csv_profiles("../data/unified/priors/T_cosmo7+24+30_mean.csv")

T_clim = read_csv_mean("../data/unified/priors/T_rasoclim_mean.csv")

T_raso_offset = T_raso.copy()
T_raso_offset.index = (T_raso.index + dt.timedelta(hours=24))
T_persistence = T_raso_offset.reindex(T_raso.index, method="nearest", tolerance=dt.timedelta(hours=3)).dropna()

In [ ]:
q_raso = np.exp(read_csv_profiles("../data/unified/test/lnq_rasoclim.csv"))

q_0006 = np.exp(read_csv_profiles("../data/unified/priors/lnq_cosmo7+00+06_mean.csv"))
q_2430 = np.exp(read_csv_profiles("../data/unified/priors/lnq_cosmo7+24+30_mean.csv"))

q_clim = np.exp(read_csv_mean("../data/unified/priors/lnq_rasoclim_mean.csv"))

q_raso_offset = q_raso.copy()
q_raso_offset.index = (q_raso.index + dt.timedelta(hours=24))
q_persistence = q_raso_offset.reindex(q_raso.index, method="nearest", tolerance=dt.timedelta(hours=3)).dropna()

## Statistical Performance

In [ ]:
fig, (axT, axq) = plt.subplots(1, 2, figsize=[8, 4.2])

statistical_eval(axT, T_raso,
         T_0006,
         T_2430,
         T_clim,
         T_persistence,
         labels=["COSMO-7 +00+06", "COSMO-7 +24+30", "Climatology", "Persistence"],
         colors=["#000000", "#666666", "#1f78b4", "#33a02c"],
         bias=False)

statistical_eval(axq, q_raso*1000,
         q_0006*1000,
         q_2430*1000,
         q_clim*1000,
         q_persistence*1000,
         colors=["#000000", "#666666", "#1f78b4", "#33a02c"],
         labels=["COSMO-7 +00+06", "COSMO-7 +24+30", "Climatology", "Persistence"],
         bias=False)

axT.set_title("statistical evaluation", loc="left", size=11)
axq.set_title("std", loc="right", size=11)
axT.set_xlabel("temperature [K]")
axq.set_xlabel("specific water content [g/kg]")
axT.set_ylim(0, 12)
axq.set_ylim(0, 12)
axq.set_ylabel("")
axq.legend(loc="upper right", fontsize=11)
fig.tight_layout()
fig.savefig("../tex/figures/retrieval_baseline.pdf")

In [ ]:
len(T_persistence)